In [1]:
import pandas as pd
import os
import datetime
os.getcwd()

'/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/Extract_1_rural_TA_with_1_store'

In [2]:
ExcelFile=pd.ExcelFile('/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx')
ExcelFile.sheet_names


['unique_zips_full_footprint',
 'view_by_TA',
 'view_by_store',
 'view_for_Tableau']

In [3]:
df_by_store=ExcelFile.parse('view_by_store',dtype=str)
df_by_TA=ExcelFile.parse('view_by_TA',dtype=str)

In [7]:
df_by_store_single=df_by_store[df_by_store['ta_name'].apply(lambda x: x[0]=="s")]
df_by_store_single.shape

(518, 17)

In [14]:
View_TA_zip_dup=pd.DataFrame()

for ind,row in df_by_store.iterrows():
    store=row['location_id']
    P_70_zips=eval(row['trans_P_zips_70_within_TA'])
    S_70_zips=eval(row['trans_S_zips_70_within_TA'])
    zips_in_10=eval(row['zips_in_10'])
    df_P=pd.DataFrame({"zips":P_70_zips},index=["P"]*len(P_70_zips)).reset_index().rename(columns={"index":"trans_flag"})
    df_S=pd.DataFrame({"zips":S_70_zips},index=["S"]*len(S_70_zips)).reset_index().rename(columns={"index":"trans_flag"})
    df_10=pd.DataFrame({"zips":zips_in_10},index=["10"]*len(zips_in_10)).reset_index().rename(columns={"index":"trans_flag"})
    
    df_store=df_P.append(df_S).append(df_10)
    df_store['location_id']=store
    View_TA_zip_dup=View_TA_zip_dup.append(df_store)
    

In [18]:
example_output=pd.DataFrame()
for store,df_group in View_TA_zip_dup.groupby("location_id"):
    copy_df=View_TA_zip_dup.copy()
    copy_df=copy_df[copy_df['location_id']!=store]
    
    set_df_group_zips=set(df_group['zips'].unique().tolist())
    set_copy_df_zips=set(copy_df['zips'].unique().tolist())
    
    if len(set_df_group_zips.intersection(set_copy_df_zips))==0:
        example_output=example_output.append(df_group)
        break

In [20]:
store_list_info=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190901-135942-508.txt",sep="|",dtype=str)


In [24]:
example_output.head(2)

,trans_flag,zips,location_id,location_desc,open_dt,close_dt,address_line_1,address_line_2,city_nm,state_nm,zip_cd,longitude_meas,latitude_meas,sales_square_feet,furniture_sales_square_feet
0,P,38930,1021,HIGHLAND PARK SHOPPING CENTER,1995-06-22,NaN,701 W PARK AVE,NaN,GREENWOOD,MS,38930-2909,-000000000090.195702,000000000033.530068,00000000022299.0000,00000000001917.0000
1,S,38751,1021,HIGHLAND PARK SHOPPING CENTER,1995-06-22,NaN,701 W PARK AVE,NaN,GREENWOOD,MS,38930-2909,-000000000090.195702,000000000033.530068,00000000022299.0000,00000000001917.0000


In [25]:
df_by_store[df_by_store['location_id']=="1021"]

,TA_num,ta_name,location_id,address_line_1,address_line_2,zip_cd,city_nm,state_nm,latitude_meas,longitude_meas,DMA,CTY,all_trans_P_zips,all_trans_S_zips,zips_in_10,trans_P_zips_70_within_TA,trans_S_zips_70_within_TA
1243,542,single_1021,1021,701 W PARK AVE,nan,38930,GREENWOOD,MS,33.530068,-90.195702,{'GREENWOOD-GREENVILLE'},"{'CARROLL', 'LEFLORE'}",['38930'],"['38941', '38917', '39169', '38954', '38751', ...",[],['38930'],"['38751', '39169', '39038', '38923', '38901', ..."


In [26]:
df_by_TA[df_by_TA['ta_name']=="single_1021"]

,TA_num,ta_name,store_list,trans_P_zips,trans_S_zips,distance_10_zips,store_count
541,542,single_1021,['1021'],['38930'],"['38701', '38751', '38901', '38917', '38923', ...",nan,1


In [21]:
example_output=pd.merge(example_output,store_list_info,on="location_id",how="left")

In [27]:
writer=pd.ExcelWriter("./BL_example_one_single_store_TA_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
example_output.to_excel(writer,"example",index=False)
df_by_store[df_by_store['location_id']=="1021"].to_excel(writer,"row_in_TA_file_store",index=False)
df_by_TA[df_by_TA['ta_name']=="single_1021"].to_excel(writer,"row_in_TA_file_TA",index=False)
writer.save()